# Codes web scraping
## NewsCounts

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

You have to download [chromedriver](https://www.selenium.dev/documentation/en/webdriver/driver_requirements/) and store in "pathToChromedriver" the path to that file.

In [5]:
pathToChromedriver = "/home/bernardo/Downloads/chromedriver"
driver = webdriver.Chrome(pathToChromedriver)
driver2 = webdriver.Chrome(pathToChromedriver)

Open [FindLaw website](https://codes.findlaw.com/) and click in the state of your choosing. Input in the search box the terms you wan to search for. I used "census population". Click search, and when the browser reloads copy the url and paste it bellow.

In [6]:
url = "https://codes.findlaw.com/LCsearch.html#?cludoquery=census%20population&cludoCategory=Codes&cludoRegion=Minnesota&cludopage=1&cludorefurl=https%3A%2F%2Fcodes.findlaw.com%2Fmn%2F&cludorefpt=Minnesota%20Statutes%20%7C%20FindLaw"
driver.get(url)

The following code will open all of the search results and store the text for each code. It will also go through all of the pages of the search result and do the same thing.

In [7]:
codes = []
page = 1
content = driver.page_source
soup = BeautifulSoup(content)
next_len = len(soup.findAll("a",attrs={"data-page":f"{page+1}"}))

while page==1 or next_len>0:
    if page>1:
        right_arrow =  driver.find_element(By.CSS_SELECTOR,f"a[data-page='{page}']").click()
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "li.search-results-item"))
            )
        finally:
            content = driver.page_source
            soup = BeautifulSoup(content)
            next_len = len(soup.findAll("a",attrs={"data-page":f"{page+1}"}))
        
    for a in soup.findAll('li',attrs={"class":"search-results-item"}):
        b= a.findNext('a',href=True)
        
        driver2.get(b["href"])
        try:
            element = WebDriverWait(driver2, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='search']"))
            )
        finally:
            content2 = driver2.page_source
            soup2 = BeautifulSoup(content2)
            text =[]

            if "404 Error" not in driver2.title:
                div_content = soup2.findAll('div',attrs={"class":"leafContent section"})
                if len(div_content)>0:
                    c =div_content[0].findAll("div",attrs={'class':'subsection'})
                    for d in c:
                        text.append(d.get_text())

            text = "\n".join(text)
            codes.append({
                "title": b["title"],
                "href": b["href"],
                "text": text
            })
   
    page=page+1
    

The results are now store in codes. We create a DataFrame out of that object.

In [10]:
df = pd.DataFrame.from_records(codes)
df

,title,href,text
0,Minnesota Statutes Evidence (Ch. 595-603) § 60...,https://codes.findlaw.com/mn/evidence-ch-595-6...,
1,Minnesota Statutes Metropolitan Area (Ch. 473-...,https://codes.findlaw.com/mn/metropolitan-area...,\n(a) The Metropolitan Council shall annually ...
2,Minnesota Statutes Property Taxes (Ch. 272-289...,https://codes.findlaw.com/mn/property-taxes-ch...,
3,"Minnesota Statutes Counties, County Officers, ...",https://codes.findlaw.com/mn/counties-county-o...,
4,Minnesota Statutes Constitutional Offices and ...,https://codes.findlaw.com/mn/constitutional-of...,\n(a) The commissioner shall appoint a state d...
...,...,...,...
624,Minnesota Statutes Criminal Procedure. Peace O...,https://codes.findlaw.com/mn/criminal-procedur...,\nSubdivision 1. Public policy. The legislatu...
625,Minnesota Statutes Property Taxes (Ch. 272-289...,https://codes.findlaw.com/mn/property-taxes-ch...,\nSubdivision 1. How classified. All real and...
626,Minnesota Statutes Public Welfare and Related ...,https://codes.findlaw.com/mn/public-welfare-an...,\nSubdivision 1. Application. The payment met...
627,Minnesota Statutes Property Taxes (Ch. 272-289...,https://codes.findlaw.com/mn/property-taxes-ch...,\n\nSubdivision 1. General rule. (a) Resident...


Finally, we store the data in a csv.

In [11]:
df.to_csv("Data/Minnesota.csv",index=False)